In [3]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm

print(f"Running on PyMC3 v{pm.__version__}")

Running on PyMC3 v3.11.2


In [4]:
#df = pd.read_csv('/Users/sletkeman/Downloads/HW5/hadlock.csv')
df = pd.read_csv("./train.csv")
df.head()

,Unnamed: 0,id,car_name,brand,model,new_price,min_cost_price,max_cost_price,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,5171,5170,Mahindra Scorpio,Mahindra,Scorpio,New Car (On-Road Price) : Rs.16.30-20.96 Lakh*,1630000.0,2096000.0,7,136542,Dealer,Diesel,Manual,15.40,2179,120.0,7,725000
1,11182,11181,Honda WR-V,Honda,WR-V,New Car (On-Road Price) : Rs.11.10-12.55 Lakh*,1110000.0,1255000.0,4,30000,Individual,Diesel,Manual,25.50,1498,98.6,5,750000
2,1425,1424,Tata Tiago,Tata,Tiago,New Car (On-Road Price) : Rs.5.63-7.89 Lakh*,563000.0,789000.0,2,15000,Individual,Petrol,Manual,23.84,1199,84.0,5,570000
3,16959,16958,Maruti S-Cross,Maruti,S-Cross,NaN,NaN,NaN,2,30000,Individual,Diesel,Manual,25.10,1248,88.5,5,1000000
4,17931,17930,Volkswagen Vento,Volkswagen,Vento,New Car (On-Road Price) : Rs.10.47-15.83 Lakh*,1047000.0,1583000.0,8,80000,Individual,Petrol,Manual,15.04,1598,103.6,5,350000


In [14]:
model = pm.Model()

with model:

    # Priors for unknown model parameters
    beta = pm.Normal("beta", mu=0, sigma=0.00001, shape=2)
    tau = pm.Gamma("tau", alpha=0.001, beta=0.001)

    # Expected value of outcome
    mu = beta[0] + beta[1] * np.sqrt(df['MA'].values)

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("BPD_obs", mu=mu, tau=tau, observed=df['BPD'].values)

In [17]:
SAMPLES = 1000
BURN = 100
with model:
    step = pm.Metropolis()
    trace = pm.sample(SAMPLES, step, chains=2, tune=BURN)
    display(az.summary(trace, round_to=2))

<ipython-input-17-cc9b10d50739>:5: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(SAMPLES, step, chains=2, tune=BURN)
Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [beta]


Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 69 seconds.
c:\Users\Scott\Bayesian Project\venv\lib\site-packages\arviz\stats\diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
The estimated number of effective samples is smaller than 200 for some parameters.
c:\Users\Scott\Bayesian Project\venv\lib\site-packages\arviz\stats\diagnostics.py:561: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],0.0,0.0,0.0,0.0,0.0,0.0,2000.00,2000.00,NaN
beta[1],0.0,0.0,0.0,0.0,0.0,0.0,2000.00,2000.00,NaN
tau,0.0,0.0,0.0,0.0,0.0,0.0,133.57,109.51,1.02
